### Deep Network[ 4 Layers Neural Network for Iris Data]

In [1]:
import json
import matplotlib
import warnings
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_auc_score

warnings.simplefilter("ignore")
root = r'C:\\Users\\admin\\Desktop\\Python_Prog\\PyMC3\\Styles\\bmh_matplotlibrc.json'
s = json.load(open(root))
matplotlib.rcParams.update(s)
% matplotlib inline

In [2]:
headers = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
iris = pd.read_csv("C:\\Users\\admin\\Desktop\\Deep Learning\\data\\iris.csv", names = headers)

iris.sepal_length =(( iris.sepal_length  - iris.sepal_length.mean())/ iris.sepal_length.std()).astype(float)
iris.sepal_width =(( iris.sepal_width  - iris.sepal_width.mean())/ iris.sepal_width.std()).astype(float)
iris.petal_length =(( iris.petal_length - iris.petal_length.mean())/ iris.petal_length.std()).astype(float)
iris.sepal_length =(( iris.petal_width  - iris.petal_width.mean())/ iris.petal_width.std()).astype(float)

In [3]:
#split our data into train, validation and test sets
#before split always remenber to chech if the class are well balance
np.random.seed(1) #for debugging we want to have the same state
shuffled_index = np.random.permutation(iris.index)
shuffled_iris = iris.loc[shuffled_index]

shuffled_iris.species = (shuffled_iris.species == 'Iris-versicolor').values.astype(int)
shuffled_iris.loc[:, ('not_species')] = shuffled_iris['species'] == 0
shuffled_iris.loc[:, ('not_species')] = shuffled_iris['not_species'].astype(int)

In [4]:
X = shuffled_iris.loc[:, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].as_matrix()
y = shuffled_iris.loc[:, ['species', 'not_species']].as_matrix()
trainX, trainY = X[:50, :], y[:50, :]
validX, validY = X[50:100], y[50:100]
testX, testY = X[100:], y[100:]

In [5]:
def accuracy(predictions, labels):
    return 100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/len(predictions)

In [6]:
hidden_units_1 = 1024
hidden_units_2 = 1024
batch_size = 40
Lambda = 1e-5
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, X.shape[1]))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, y.shape[1]))
    tf_valid_dataset = tf.placeholder(tf.float32, shape=(validX.shape))
    tf_test_dataset = tf.placeholder(tf.float32, shape=(testX.shape))
    global_step = tf.Variable(0)
    
    #Variables
    weights1 = tf.Variable(tf.truncated_normal([X.shape[1], hidden_units_1], 
                                               stddev = np.sqrt(2.0 / X.shape[1])))
    biases1 = tf.Variable(tf.zeros([hidden_units_1]))
    weights2 = tf.Variable(tf.truncated_normal([hidden_units_1, hidden_units_2],
                                               stddev = np.sqrt(2.0 / hidden_units_1)))
    biases2 = tf.Variable(tf.zeros([hidden_units_2]))
    weights3 = tf.Variable(tf.truncated_normal([hidden_units_2, y.shape[1]],
                                               stddev = np.sqrt(2.0 / hidden_units_2)))
    biases3 = tf.Variable(tf.zeros([y.shape[1]]))
    
    #Training
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    logits = tf.matmul(hidden2, weights3) + biases3
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))\
    + (Lambda/(2*batch_size))*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
    
    #Optmization
    learning_rate = tf.train.exponential_decay(0.8, global_step, 4000, 0.6, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    #Predictions
    train_predictions = tf.nn.softmax(logits)
    hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    hidden_valid_2 = tf.nn.relu(tf.matmul(hidden_valid_1, weights2) + biases2)
    valid_predictions = tf.nn.softmax(tf.matmul(hidden_valid_2, weights3) + biases3)
    hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1, weights2) + biases2)
    test_predictions = tf.nn.softmax(tf.matmul(hidden_test_2, weights3) + biases3)

In [7]:
#train
num_steps = 5001
with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in np.arange(num_steps):
        offset = (step * batch_size) % (trainY.shape[0]- batch_size)
        batch_data = trainX[offset:(offset + batch_size), :]
        batch_labels = trainY[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict = feed_dict)
        if step%1000 == 0:
            print("Minibatch loss at {0}: {1}".format(step, l))
            print("Minibatch accuray : %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                        valid_predictions.eval(feed_dict={tf_valid_dataset:validX}), validY))
    print("Test accuracy: %.1f%%" % accuracy(test_predictions.eval(feed_dict={tf_test_dataset:testX}), 
                                             testY))

Initialized
Minibatch loss at 0: 0.677087128162384
Minibatch accuray : 57.5%
Validation accuracy: 46.0%
Minibatch loss at 1000: 0.0012525484198704362
Minibatch accuray : 100.0%
Validation accuracy: 92.0%
Minibatch loss at 2000: 0.0012229284038767219
Minibatch accuray : 100.0%
Validation accuracy: 92.0%
Minibatch loss at 3000: 0.0012144912034273148
Minibatch accuray : 100.0%
Validation accuracy: 92.0%
Minibatch loss at 4000: 0.0012108426308259368
Minibatch accuray : 100.0%
Validation accuracy: 92.0%
Minibatch loss at 5000: 0.001209388137795031
Minibatch accuray : 100.0%
Validation accuracy: 92.0%
Test accuracy: 96.0%
